In [17]:
!pip install dask

In [18]:
import dask.dataframe as dd
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [19]:
#wczytanie pliku CSV
dask_df = dd.read_csv("bigdata.csv")

dask_df.head(3)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,label
0,-1.871161,1.267846,-0.788753,-1.078589,-7.674173,1.898069,-1.047163,5.716543,-1.888746,2.728939,-2.666232,0.872623,-2.871979,0.767160,11.712507,-10.955615,0.851592,1.816229,-2.179645,0.702227,0
1,3.704016,-2.702605,1.727635,-1.734933,2.624712,-2.332045,1.984392,-0.830701,-1.942123,-0.776450,1.600891,0.601621,-0.367161,-0.113178,-6.191234,3.449323,3.284086,-1.262817,2.929419,-0.075368,0
2,3.659057,-3.089243,0.087034,-3.057691,2.080943,-0.856224,-0.679463,-0.435035,0.610472,0.027802,-2.981704,0.309686,3.745178,-0.638586,-1.184934,2.952795,-2.092829,2.965016,-3.595892,0.261711,0


In [20]:
#podziałna cechy
X_dask = dask_df.drop("label",axis=1)
y_dask = dask_df["label"]

In [21]:
#skalowanie (normalizacja) danych
sample = X_dask.sample(frac=0.2).compute()
scaler = StandardScaler()
scaler.fit(sample)

StandardScaler() In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. StandardScaler ? Documentation for StandardScaler i Fitted StandardScaler()

In [22]:
clf = SGDClassifier(loss='hinge',max_iter = 15)

In [15]:
#uczenie partiami
for epoch in range(10):
    for X_chunk,y_chunk in zip(X_dask.to_delayed(),y_dask.to_delayed()):
        X_np = scaler.transform(X_chunk.compute())
        y_np = y_chunk.compute()
        clf.partial_fit(X_np,y_np,classes=np.array([0,1]))

In [16]:
#ocena modelu
from sklearn.datasets import make_classification
X_test,y_test = make_classification(n_samples=5000,n_features=20,n_informative=15,
                                    n_redundant=5,random_state=123)

X_test_scaled = scaler.transform(X_test)
y_pred = clf.predict(X_test_scaled)

print(f"Dokładność modelu: {accuracy_score(y_test,y_pred):.4f}")

Dokładność modelu: 0.4774


/opt/python/envs/default_3_11/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
